In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window as W

spark = SparkSession.builder.appName('Puzzle').getOrCreate()

In [11]:
schema = T.StructType(
    [
    T.StructField('PatientID',T.IntegerType(), True),
    T.StructField('AdmissionDate',T.DateType(), True),
    T.StructField('DischargeDate',T.DateType(), True),
    T.StructField('Cost',T.IntegerType(), True)
    ]
)

df = spark.read.csv('data/puzzle_030.csv', header=True, schema=schema, dateFormat="dd-MM-yyyy")
df.show()
df.printSchema()

+---------+-------------+-------------+----+
|PatientID|AdmissionDate|DischargeDate|Cost|
+---------+-------------+-------------+----+
|     1009|   2014-07-27|   2014-07-31|1050|
|     1009|   2014-08-01|   2014-08-23|1070|
|     1009|   2014-08-31|   2014-08-31|1900|
|     1009|   2014-09-01|   2014-09-14|1260|
|     1009|   2014-12-01|   2014-12-31|2090|
|     1024|   2014-06-07|   2014-06-28|1900|
|     1024|   2014-06-29|   2014-07-31|2900|
|     1024|   2014-08-01|   2014-08-02|1800|
+---------+-------------+-------------+----+

root
 |-- PatientID: integer (nullable = true)
 |-- AdmissionDate: date (nullable = true)
 |-- DischargeDate: date (nullable = true)
 |-- Cost: integer (nullable = true)



In [16]:
df.select(
    '*',
    F.lag(F.col('DischargeDate'),1, '1972-01-01').over(W.orderBy(F.lit(1))).alias('PRvDschDt')
).select(
    '*',
    F.sum(F.when(
        F.datediff(F.col('AdmissionDate'), F.col('PRvDschDt')) != 1, 1
        ).otherwise(0)).over(W.orderBy('PatientID', 'AdmissionDate')).alias('Grp')
).groupBy("PatientID", 'Grp').agg(
    F.min(F.col('AdmissionDate')).alias('AdmissionDate'),
    F.max(F.col('DischargeDate')).alias('DischargeDate'),
    F.sum(F.col('Cost')).alias('Cost')).show()
    


+---------+---+-------------+-------------+----+
|PatientID|Grp|AdmissionDate|DischargeDate|Cost|
+---------+---+-------------+-------------+----+
|     1009|  1|   2014-07-27|   2014-08-23|2120|
|     1009|  2|   2014-08-31|   2014-09-14|3160|
|     1009|  3|   2014-12-01|   2014-12-31|2090|
|     1024|  4|   2014-06-07|   2014-08-02|6600|
+---------+---+-------------+-------------+----+

